<a href="https://colab.research.google.com/github/BeataStultica/lstm_labs/blob/main/laba8_cnn_bi_lstm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!tar -xvf '/content/gdrive/MyDrive/train-clean-100.tar.gz' -C '/content'

Показано результат, скорочений до останніх рядків (5000).
LibriSpeech/train-clean-100/2136/5143/2136-5143-0024.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0012.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0044.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0003.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0010.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0025.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0043.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0004.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0026.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0015.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0032.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0027.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0039.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0020.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0023.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0007.flac
LibriSpeech/tr

In [3]:
char_map_str = """
' 0
<SPACE> 1
a 2
b 3
c 4
d 5
e 6
f 7
g 8
h 9
i 10
j 11
k 12
l 13
m 14
n 15
o 16
p 17
q 18
r 19
s 20
t 21
u 22
v 23
w 24
x 25
y 26
z 27
"""


char_map = {}
index_map = {}
for line in char_map_str.strip().split('\n'):
    ch, index = line.split()
    char_map[ch] = int(index)
    index_map[int(index)+1] = ch  
index_map[2] = ' '

In [5]:
!sudo ln -s /usr/bin/ffmpeg /usr/bin/avconv

In [6]:
%%writefile flac_to_wav2.sh

find . -iname "*.flac" | wc
for flacfile in `find . -iname "*.flac"`
do
    avconv -y -f flac -i $flacfile -ab 64k -ac 1 -ar 16000 -f wav "${flacfile%.*}.wav"
done

Writing flac_to_wav2.sh


In [9]:
!chmod 755 /content/LibriSpeech/flac_to_wav2.sh

In [10]:
! /content/LibriSpeech/flac_to_wav2.sh

Показано результат, скорочений до останніх рядків (5000).
Output #0, wav, to './LibriSpeech/train-clean-100/5192/19397/5192-19397-0031.wav':
  Metadata:
    ISFT            : Lavf57.83.100
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 16000 Hz, mono, s16, 256 kb/s
    Metadata:
      encoder         : Lavc57.107.100 pcm_s16le
size=     483kB time=00:00:15.46 bitrate= 256.0kbits/s speed=3.05e+03x    
video:0kB audio:483kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.015761%
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflit

In [11]:
from __future__ import absolute_import, division, print_function

import argparse
import json
import os
import wave
def main(data_directory, output_file):
    labels = []
    durations = []
    keys = []
    for group in os.listdir(data_directory):
        if group.startswith('.'):
            continue
        speaker_path = os.path.join(data_directory, group)
        for speaker in os.listdir(speaker_path):
            if speaker.startswith('.'):
                continue
            labels_file = os.path.join(speaker_path, speaker,
                                       '{}-{}.trans.txt'
                                       .format(group, speaker))
            for line in open(labels_file):
                split = line.strip().split()
                file_id = split[0]
                label = ' '.join(split[1:]).lower()
                audio_file = os.path.join(speaker_path, speaker,
                                          file_id) + '.wav'
                audio = wave.open(audio_file)
                duration = float(audio.getnframes()) / audio.getframerate()
                audio.close()
                keys.append(audio_file)
                durations.append(duration)
                labels.append(label)
    with open(output_file, 'w') as out_file:
        for i in range(len(keys)):
            line = json.dumps({'key': keys[i], 'duration': durations[i],
                              'text': labels[i]})
            out_file.write(line + '\n')
main('/content/LibriSpeech/train-clean-100', 'train_100.json')

In [ ]:
!tar -czvf "/content/train_new.tar.gz" "/content/LibriSpeech/train-clean-100"

Показано результат, скорочений до останніх рядків (5000).
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0009.wav
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0006.flac
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0015.wav
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0000.wav
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0006.wav
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0029.wav
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0019.flac
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0034.flac
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0019.wav
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0022.wav
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0017.wav
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0016.flac
/content/LibriSpeech/train-clean-100/7059/88364/7059-88364-0028.wav
/content/LibriSpeech/train-clean-100/7059/88364/7059-8

In [37]:
import numpy as np
import soundfile

def calc_feat_dim(window, max_freq):
    return int(0.001 * window * max_freq) + 1

def text_to_int_sequence(text):
    int_sequence = []
    for c in text:
        if c == ' ':
            ch = char_map['<SPACE>']
        else:
            ch = char_map[c]
        int_sequence.append(ch)
    return int_sequence

def int_sequence_to_text(int_sequence):
    text = []
    print(index_map)
    for c in int_sequence:
        if c==0:
          continue
        else:
          ch = index_map[c]
          text.append(ch)
    return text

def spectrogram(samples, fft_length=256, sample_rate=2, hop_length=128):
    window = np.hanning(fft_length)[:, None]
    window_norm = np.sum(window**2)

    scale = window_norm * sample_rate

    trunc = (len(samples) - fft_length) % hop_length
    x = samples[:len(samples) - trunc]

    nshape = (fft_length, (len(x) - fft_length) // hop_length + 1)
    nstrides = (x.strides[0], x.strides[0] * hop_length)
    x = np.lib.stride_tricks.as_strided(x, shape=nshape, strides=nstrides)
    assert np.all(x[:, 1] == samples[hop_length:(hop_length + fft_length)])
    x = np.fft.rfft(x * window, axis=0)
    x = np.absolute(x)**2
    x[1:-1, :] *= (2.0 / scale)
    x[(0, -1), :] /= scale

    freqs = float(sample_rate) / fft_length * np.arange(x.shape[0])

    return x, freqs

def soundfile_load(filename):
    with soundfile.SoundFile(filename) as sound_file:
        audio = sound_file.read(dtype='float32')
        sample_rate = sound_file.samplerate
    return audio,sample_rate

def spectrogram_from_file(filename, step=10, window=20, max_freq=None,
                          eps=1e-14):
    audio,sample_rate = soundfile_load(filename)
    if audio.ndim >= 2:
        audio = np.mean(audio, 1)
    if max_freq is None:
        max_freq = sample_rate / 2
    if max_freq > sample_rate / 2:
        raise ValueError("max_freq must not be greater than half of "
                            " sample rate")
    if step > window:
        raise ValueError("step size must not be greater than window size")
    hop_length = int(0.001 * step * sample_rate)
    fft_length = int(0.001 * window * sample_rate)
    pxx, freqs = spectrogram(
        audio, fft_length=fft_length, sample_rate=sample_rate,
        hop_length=hop_length)
    ind = np.where(freqs <= max_freq)[0][-1] + 1
    return np.transpose(np.log(pxx[:ind, :] + eps))

def sort_data(audio_paths, durations, texts):
    p = np.argsort(durations).tolist()
    audio_paths = [audio_paths[i] for i in p]
    durations = [durations[i] for i in p] 
    texts = [texts[i] for i in p]
    return audio_paths, durations, texts

def shuffle_data(audio_paths, durations, texts):
    p = np.random.permutation(len(audio_paths))
    audio_paths = [audio_paths[i] for i in p] 
    durations = [durations[i] for i in p] 
    texts = [texts[i] for i in p]
    return audio_paths, durations, texts

In [29]:
import json
import numpy as np
import random
import scipy.io.wavfile as wav
from itertools import islice
RNG_SEED = 123

class AudioGenerator():
    def __init__(self, step=10, window=20, max_freq=8000, mfcc_dim=13,
        minibatch_size=20, desc_file=None, spectrogram=True, max_duration=10.0, 
        sort_by_duration=False):

        self.feat_dim = calc_feat_dim(window, max_freq)
        self.mfcc_dim = mfcc_dim
        self.feats_mean = np.zeros((self.feat_dim,))
        self.feats_std = np.ones((self.feat_dim,))
        self.rng = random.Random(RNG_SEED)
        if desc_file is not None:
            self.load_metadata_from_desc_file(desc_file)
        self.step = step
        self.window = window
        self.max_freq = max_freq
        self.cur_train_index = 0
        self.cur_valid_index = 0
        self.cur_test_index = 0
        self.max_duration=max_duration
        self.minibatch_size = minibatch_size
        self.spectrogram = spectrogram
        self.sort_by_duration = sort_by_duration

    def get_batch(self, partition):
        if partition == 'train':
            audio_paths = self.train_audio_paths
            cur_index = self.cur_train_index
            texts = self.train_texts
        elif partition == 'valid':
            audio_paths = self.valid_audio_paths
            cur_index = self.cur_valid_index
            texts = self.valid_texts
        else:
            raise Exception("Invalid partition. "
                "Must be train/validation")

        features = [self.normalize(self.featurize(a)) for a in 
            audio_paths[cur_index:cur_index+self.minibatch_size]]

        max_length = max([features[i].shape[0] 
            for i in range(0, self.minibatch_size)])
        max_string_length = max([len(texts[cur_index+i]) 
            for i in range(0, self.minibatch_size)])
        

        X_data = np.zeros([self.minibatch_size, max_length, 
            self.feat_dim*self.spectrogram + self.mfcc_dim*(not self.spectrogram)])
        labels = np.ones([self.minibatch_size, max_string_length]) * 28
        input_length = np.zeros([self.minibatch_size, 1])
        label_length = np.zeros([self.minibatch_size, 1])
        
        for i in range(0, self.minibatch_size):
            feat = features[i]
            input_length[i] = feat.shape[0]
            X_data[i, :feat.shape[0], :] = feat

            label = np.array(text_to_int_sequence(texts[cur_index+i])) 
            labels[i, :len(label)] = label
            label_length[i] = len(label)
 
        outputs = {'ctc': np.zeros([self.minibatch_size])}
        inputs = {'the_input': X_data, 
                  'the_labels': labels, 
                  'input_length': input_length, 
                  'label_length': label_length 
                 }
        return (inputs, outputs)

 
    def next_train(self):
        while True:
            ret = self.get_batch('train')
            self.cur_train_index += self.minibatch_size
            if self.cur_train_index >= len(self.train_texts) - self.minibatch_size:
                self.cur_train_index = 0
                self.shuffle_data_by_partition('train')
            yield ret    

    def next_valid(self):
        while True:
            ret = self.get_batch('valid')
            self.cur_valid_index += self.minibatch_size
            if self.cur_valid_index >= len(self.valid_texts) - self.minibatch_size:
                self.cur_valid_index = 0
                self.shuffle_data_by_partition('valid')
            yield ret


    def load_metadata_from_desc_file(self, desc_file, partition ='train'):
        audio_paths, durations, texts = [], [], []
        start = 0
        end = 24539
        if partition =='validation':
          start = end
          end = 28539
        with open(desc_file) as json_line_file:
            for line_num, json_line in islice(enumerate(json_line_file), start, end):

                try:
                    spec = json.loads(json_line)
                    if float(spec['duration']) > self.max_duration:
                        continue
                    audio_paths.append(spec['key'])
                    durations.append(float(spec['duration']))
                    texts.append(spec['text'])
                except Exception as e:
                    print('Error reading line #{}: {}'
                                .format(line_num, json_line))
        if partition == 'train':
            self.train_audio_paths = audio_paths
            self.train_durations = durations
            self.train_texts = texts
        elif partition == 'validation':
            self.valid_audio_paths = audio_paths
            self.valid_durations = durations
            self.valid_texts = texts
        else:
            raise Exception("Invalid partition to load metadata. "
             "Must be train/validation")
    def load_train_data(self, desc_file='train_100.json'):
        self.load_metadata_from_desc_file(desc_file, 'train')
        self.fit_train()
        if self.sort_by_duration:
            self.sort_data_by_duration('train')
    def load_validation_data(self, desc_file='train_100.json'):
        self.load_metadata_from_desc_file(desc_file, 'validation')
        if self.sort_by_duration:
            self.sort_data_by_duration('valid')
   
    def fit_train(self, k_samples=100):
        k_samples = min(k_samples, len(self.train_audio_paths))
        samples = self.rng.sample(self.train_audio_paths, k_samples)
        feats = [self.featurize(s) for s in samples]
        feats = np.vstack(feats)
        self.feats_mean = np.mean(feats, axis=0)
        self.feats_std = np.std(feats, axis=0)
        
    def featurize(self, audio_clip_path):

        return spectrogram_from_file(
                audio_clip_path, step=self.step, window=self.window,
                max_freq=self.max_freq)

    def normalize(self, feature, eps=1e-14):
        return (feature - self.feats_mean) / (self.feats_std + eps)

    def shuffle_data_by_partition(self, partition):
        if partition == 'train':
            self.train_audio_paths, self.train_durations, self.train_texts = shuffle_data(
                self.train_audio_paths, self.train_durations, self.train_texts)
        elif partition == 'valid':
            self.valid_audio_paths, self.valid_durations, self.valid_texts = shuffle_data(
                self.valid_audio_paths, self.valid_durations, self.valid_texts)
        else:
            raise Exception("Invalid partition. "
                "Must be train/validation")

    def sort_data_by_duration(self, partition):
        if partition == 'train':
            self.train_audio_paths, self.train_durations, self.train_texts = sort_data(
                self.train_audio_paths, self.train_durations, self.train_texts)
        elif partition == 'valid':
            self.valid_audio_paths, self.valid_durations, self.valid_texts = sort_data(
                self.valid_audio_paths, self.valid_durations, self.valid_texts)
        else:
            raise Exception("Invalid partition. "
                "Must be train/validation")

In [27]:
import numpy as np
from tensorflow.keras import backend as K



class ModelPredictor:

    def __init__(self,input_to_softmax=None,model_path='/content/gdrive/MyDrive/cnn_bi_lstm.h5'):
        self.input_to_softmax = input_to_softmax
        self.input_to_softmax.load_weights(model_path) #keras.models.load_model(model_path)
        self.audio_path = None
        
    def get_predictions_index(self,index, partition, spectrogram=True):
        data_gen = AudioGenerator(spectrogram=spectrogram)
        data_gen.load_train_data()
        data_gen.load_validation_data()

        if partition == 'validation':
            transcr = data_gen.valid_texts[index]
            self.audio_path = data_gen.valid_audio_paths[index]
            data_point = data_gen.normalize(data_gen.featurize(self.audio_path))
        elif partition == 'train':
            transcr = data_gen.train_texts[index]
            self.audio_path = data_gen.train_audio_paths[index]
            data_point = data_gen.normalize(data_gen.featurize(self.audio_path))
        else:
            raise Exception('Invalid partition!  Must be "train" or "validation"')
        

        prediction = self.input_to_softmax.predict(np.expand_dims(data_point, axis=0))
        output_length = [self.input_to_softmax.output_length(data_point.shape[0])] 
        pred_ints = (K.eval(K.ctc_decode(
                    prediction, output_length)[0][0])+1).flatten().tolist()
        

        print('True transcription:\n' + '\n' + transcr)
        print('-'*80)
        print('Predicted transcription:\n' + '\n' + ''.join(int_sequence_to_text(pred_ints)))
        print('-'*80)

In [52]:

import _pickle as pickle
from tensorflow import keras
from tensorflow.keras import metrics
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Lambda)
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint  ,TensorBoard
from tensorflow.keras import losses 
import datetime
import os

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def add_ctc_loss(input_to_softmax):
    the_labels = Input(name='the_labels', shape=(None,), dtype='float32')
    input_lengths = Input(name='input_length', shape=(1,), dtype='int64')
    label_lengths = Input(name='label_length', shape=(1,), dtype='int64')
    output_lengths = Lambda(input_to_softmax.output_length)(input_lengths)
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')(
        [input_to_softmax.output, the_labels, output_lengths, label_lengths])
    model = Model(
        inputs=[input_to_softmax.input, the_labels, input_lengths, label_lengths], 
        outputs=loss_out)
    return model
def train_model(input_to_softmax, 
                save_model_path,
                train_json='train_100.json',
                valid_json='train_100.json',
                minibatch_size=20,
                spectrogram=True,
                mfcc_dim=13,
                optimizer='adam',
                epochs=20,
                verbose=1,
                sort_by_duration=False,
                max_duration=10.0):

    audio_gen = AudioGenerator(minibatch_size=minibatch_size, 
        spectrogram=spectrogram, mfcc_dim=mfcc_dim, max_duration=max_duration,
        sort_by_duration=sort_by_duration)
    
    audio_gen.load_train_data(train_json)
    audio_gen.load_validation_data(valid_json)

    num_train_examples=len(audio_gen.train_audio_paths)
    steps_per_epoch = num_train_examples//minibatch_size

    num_valid_samples = len(audio_gen.valid_audio_paths) 
    validation_steps = num_valid_samples//minibatch_size
    

    model = add_ctc_loss(input_to_softmax)

    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)

    checkpointer = ModelCheckpoint(filepath='/content/gdrive/MyDrive/'+save_model_path, verbose=0)

    hist = model.fit(x=audio_gen.next_train(), steps_per_epoch=steps_per_epoch,
        epochs=epochs, validation_data=audio_gen.next_valid(), validation_steps=validation_steps, callbacks=[checkpointer], verbose=verbose)

In [50]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (BatchNormalization, Conv1D, Dense, Input, 
    TimeDistributed, Activation, Bidirectional, LSTM,MaxPool1D)
from tensorflow.keras.initializers import VarianceScaling


def cnn_output_length(input_length, filter_size, border_mode, stride,
                       dilation=1):
    if input_length is None:
        return None
    assert border_mode in {'same', 'valid'}
    dilated_filter_size = filter_size + (filter_size - 1) * (dilation - 1)
    if border_mode == 'same':
        output_length = input_length
    elif border_mode == 'valid':
        output_length = input_length - dilated_filter_size + 1
    return (output_length + stride - 1) // stride
def cnn_bi_lstm_model(input_dim, filters, kernel_size, conv_stride,
    conv_border_mode, units, output_dim=29,name='cnn_bi_lstm_model'):
    input_data = Input(name='the_input', shape=(None, input_dim))
    conv_1d = Conv1D(filters, kernel_size, 
                     strides=conv_stride, 
                     padding=conv_border_mode,
                     activation='relu',
                     name='conv1d')(input_data)
    bn_cnn = BatchNormalization(name='bn_conv_1d')(conv_1d)
    layer = layers.Bidirectional(layers.LSTM(units,
      return_sequences=True))(bn_cnn)   
    bn_rnn = BatchNormalization(name='bn_rnn')(layer)
    time_dense= TimeDistributed(Dense(output_dim,activation='relu'))(bn_rnn)
    y_pred = Activation('softmax', name='softmax')(time_dense)
    model = Model(inputs=input_data, outputs=y_pred,name=name)
    model.output_length = lambda x: cnn_output_length(
        x, kernel_size, conv_border_mode, conv_stride)
    #print(model.summary())
    return model


In [22]:

model = cnn_bi_lstm_model(input_dim=161,
                        filters=200,
                        kernel_size=11, 
                        conv_stride=2,
                        conv_border_mode='valid',
                        units=200,
                        name='cnn_bi_lstm_spectrogram')
print(model.summary())
train_model(input_to_softmax=model,  
            save_model_path='cnn_bi_lstm.h5')

Model: "cnn_bi_lstm_spectrogram"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 161)]       0         
                                                                 
 conv1d (Conv1D)             (None, None, 200)         354400    
                                                                 
 bn_conv_1d (BatchNormalizat  (None, None, 200)        800       
 ion)                                                            
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 400)        641600    
 nal)                                                            
                                                                 
 bn_rnn (BatchNormalization)  (None, None, 400)        1600      
                                                                 
 time_distributed_1 (TimeDis  (None, None, 

In [48]:
model2 = cnn_bi_lstm_model(input_dim=161,
                        filters=200,
                        kernel_size=11, 
                        conv_stride=2,
                        conv_border_mode='valid',
                        units=200,
                        name='cnn_bi_lstm_spectrogram')
predictor = ModelPredictor(model2, model_path='/content/gdrive/MyDrive/cnn_bi_lstm.h5')
from IPython.display import Audio


predictor.get_predictions_index(index=1, 
                partition='validation')

Audio(predictor.audio_path)


predictor.get_predictions_index(index=18, 
                partition='validation')

Audio(predictor.audio_path)

True transcription:

the next day mister o'brien again attempted to see his clients as did also the mother of miss matilda young the youngest prisoner in mister whittaker's care
--------------------------------------------------------------------------------
{1: "'", 2: ' ', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'q', 20: 'r', 21: 's', 22: 't', 23: 'u', 24: 'v', 25: 'w', 26: 'x', 27: 'y', 28: 'z'}
Predicted transcription:

the nex dy masstera bri an agan ittented tose his cuiandce pause dod als of the mother af miss mittill the yon the youndust prisnerin mister it  cors cre
--------------------------------------------------------------------------------
True transcription:

chapter thirty four unc billy and old missus possum
--------------------------------------------------------------------------------
{1: "'", 2: ' ', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 